In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Conv2D, MaxPooling2D, Flatten, MaxPooling1D, Conv1D
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import tensorflow as tf

import tensorflow_addons as tfa
from keras.utils import pad_sequences
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [7]:
df=pd.read_csv('input10.csv')

In [8]:
df.head()

,comment,class
0,FIXME formatters are not thread safe,DEFECT
1,"XXX Jon Skeet The comment ""if it hasn't bee...",DEFECT
2,"I hate to admit it, but we don't know what h...",DEFECT
3,Just a note StarTeam has a status for NEW wh...,DEFECT
4,the generated classes must not be added in t...,DEFECT


In [10]:
df['class'].value_counts()

DESIGN            2703
IMPLEMENTATION     757
DEFECT             472
TEST                85
DOCUMENTATION       54
Name: class, dtype: int64

In [64]:
X = df['comment']
Y = df['class']
le = LabelEncoder()
Y = le.fit_transform(Y)

multiclass_labels = []
print("\nMulticlass Label Encodings (in order of digits 0 -> n): ")
for i in range(0, len(list(set(list(Y))))):
    multiclass_labels.append(le.inverse_transform([i])[0])

print(multiclass_labels)
print("\n Class weights:")
#print(np.unique(Y, return_counts = True))
weight_dict = {}
counts = np.unique(Y, return_counts = True)[1]
total = np.sum(counts)
for i in range (0, 5):
    weight_dict[i] = (total - counts[i]) / counts[i]

print(weight_dict)

Y = Y.reshape(-1,1)


Multiclass Label Encodings (in order of digits 0 -> n): 
['DEFECT', 'DESIGN', 'DOCUMENTATION', 'IMPLEMENTATION', 'TEST']

 Class weights:
{0: 7.625, 1: 0.5061043285238623, 2: 74.38888888888889, 3: 4.3778071334214, 4: 46.89411764705882}


In [65]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.20, stratify=Y, random_state=42)

max_words = 10000
max_len = 250
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
train_sequences = tok.texts_to_sequences(X_train)
train_sequences_matrix = pad_sequences(train_sequences,maxlen=max_len)

test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [66]:
print(len(train_sequences_matrix[0]))

250


In [163]:
def CNN():
    cnn = tf.keras.Sequential()
    cnn.add(Input(name='inputs',shape=[max_len]))
    cnn.add(Embedding(max_words,50,input_length=max_len))
    cnn.add(Conv1D(128, kernel_size=3, activation='relu', padding="same"))
    cnn.add(MaxPooling1D(pool_size=2))
    cnn.add(Conv1D(256, kernel_size=3, activation='relu', padding="same"))
    cnn.add(MaxPooling1D(pool_size=2))
    cnn.add(Conv1D(512, kernel_size=3, activation='relu', padding="same"))
    cnn.add(MaxPooling1D(pool_size=2))

    cnn.add(Flatten())
    cnn.add(Dense(256, activation ='relu'))
    cnn.add(Dense(64, activation ='relu'))
    cnn.add(Dense(5, activation='softmax'))

    return cnn

In [164]:
model = CNN()
model.summary()
#opt = tf.keras.optimizers.Adadelta(learning_rate=0.1)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy',optimizer=opt,metrics=['sparse_categorical_accuracy'])

Model: "sequential_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_35 (Embedding)    (None, 250, 50)           500000    
                                                                 
 conv1d_85 (Conv1D)          (None, 250, 128)          19328     
                                                                 
 max_pooling1d_85 (MaxPoolin  (None, 125, 128)         0         
 g1D)                                                            
                                                                 
 conv1d_86 (Conv1D)          (None, 125, 256)          98560     
                                                                 
 max_pooling1d_86 (MaxPoolin  (None, 62, 256)          0         
 g1D)                                                            
                                                                 
 conv1d_87 (Conv1D)          (None, 62, 512)         

In [165]:
#model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
checkpoint_filepath = './checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                filepath=checkpoint_filepath,
                save_weights_only=True,
                monitor='val_loss',
                mode='min',
                save_best_only=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5, min_lr=0.000001)

model.fit(train_sequences_matrix,Y_train,batch_size=128,epochs=50,validation_split=0.2, callbacks=[model_checkpoint_callback,reduce_lr], class_weight=weight_dict)

model.load_weights(checkpoint_filepath)

Epoch 1/50
21/21 [==============================] - 1s 36ms/step - loss: 6.1043 - sparse_categorical_accuracy: 0.0499 - val_loss: 2.1450 - val_sparse_categorical_accuracy: 0.0261 - lr: 0.0010
Epoch 2/50
21/21 [==============================] - 1s 30ms/step - loss: 5.8439 - sparse_categorical_accuracy: 0.0300 - val_loss: 2.0938 - val_sparse_categorical_accuracy: 0.1242 - lr: 0.0010
Epoch 3/50
21/21 [==============================] - 1s 31ms/step - loss: 4.9082 - sparse_categorical_accuracy: 0.1793 - val_loss: 2.0411 - val_sparse_categorical_accuracy: 0.1074 - lr: 0.0010
Epoch 4/50
21/21 [==============================] - 1s 31ms/step - loss: 3.6744 - sparse_categorical_accuracy: 0.1486 - val_loss: 1.7538 - val_sparse_categorical_accuracy: 0.2730 - lr: 0.0010
Epoch 5/50
21/21 [==============================] - 0s 23ms/step - loss: 3.1102 - sparse_categorical_accuracy: 0.1778 - val_loss: 2.0661 - val_sparse_categorical_accuracy: 0.2439 - lr: 0.0010
Epoch 6/50
21/21 [======================

In [171]:
model.save("CNN_imbalanced_multi.h5")

In [166]:
accr = model.evaluate(test_sequences_matrix,Y_test)

26/26 [==============================] - 0s 9ms/step - loss: 1.4633 - sparse_categorical_accuracy: 0.4773


In [167]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 1.463
  Accuracy: 0.477


In [168]:
y_pred = model.predict(test_sequences_matrix)

26/26 [==============================] - 0s 9ms/step


In [169]:
y_pred2 = [np.argmax(x) for x in y_pred]

In [170]:
print(classification_report(Y_test, y_pred2, digits=6, target_names=multiclass_labels))

                precision    recall  f1-score   support

        DEFECT   0.287037  0.659574  0.400000        94
        DESIGN   0.823077  0.395564  0.534332       541
 DOCUMENTATION   0.400000  0.363636  0.380952        11
IMPLEMENTATION   0.329268  0.710526  0.450000       152
          TEST   1.000000  0.058824  0.111111        17

      accuracy                       0.477301       815
     macro avg   0.567876  0.437625  0.375279       815
  weighted avg   0.667135  0.477301  0.492212       815

